## Import the libraries

In [1]:
# Standard
import numpy as np            # Handle matrices
from scipy import stats
import math

# EV data management
import data_collection        # Get EV Charging Data
import pymongo
import bson
from datetime import datetime, timedelta

# RL packages
import random                 # Handling random number generation
from random import choices
from collections import deque # Ordered collection with ends

## Collect and process real data

In [2]:
# Acquire data from database; Run once
# Database information
# For training
acn  = 'JplACN'
start = datetime(2018, 11,1)
end = datetime(2019, 4,2)
dt = timedelta(days = 1)
dates = np.arange(start, end, dt).astype(datetime)
day = 0
number_train_days= 0
for date in dates:
    # Eliminate weekends
    if date.weekday() != 5 and date.weekday() != 6:
        charging_start = date
        charging_end = date + timedelta(1)
        data_train = data_collection.get_data(acn, charging_start, charging_end, cond='ALL')
        # If at least 10 cars came
        if data_train.size>= 30:
            number_train_days = number_train_days+1
            name = 'real_train/data' + str(day) + '.npy'
            np.save(name, data_train)
            day=day+1

In [5]:
# For testing
start = datetime(2019, 4,2)
end = datetime(2019, 5,2)
dt = timedelta(days = 1)
dates = np.arange(start, end, dt).astype(datetime)
day = 0
number_test_days= 0
for date in dates:
    # Eliminate weekends
    if date.weekday() != 5 and date.weekday() != 6:
        charging_start = date
        charging_end = date + timedelta(1)
        data_test = data_collection.get_data(acn, charging_start, charging_end, cond='ALL')
        # If at least 10 cars came
        if data_test.size>= 30:
            number_test_days = number_test_days+1
            name = 'real_test/data' + str(day) + '.npy'
            np.save(name, data_test)
            day=day+1

In [6]:
# Load data
# Clean up training data
for day in range(number_train_days):
    name_train = 'real_train/data' + str(day) + '.npy'
    data_train = np.load(name_train)
    for car in range(np.size(data_train,0)):
        # Eliminate over-night cars
        if data_train[car,0] + data_train[car,1] > 24:
            np.delete(data_train,car,0)
    # Sort according to arrivial time
    data_train =  data_train[data_train[:,0].argsort()]
    np.save(name_train, data_train)

In [29]:
print(number_test_days)

22


In [7]:
# Load data
# Clean up testing data
for day in range(number_test_days):
    name_test = 'real_test/data' + str(day) + '.npy'
    data_test = np.load(name_test)
    for car in range(np.size(data_test,0)):
        # Eliminate over-night cars
        if data_test[car,0] + data_test[car,1] > 24:
            np.delete(data_test,car,0)
    # Sort according to arrivial time
    data_test =  data_test[data_test[:,0].argsort()]
    np.save(name_test, data_test)

In [2]:
# Test
for day in range(50):
    name = 'real_train/data' + str(day) + '.npy'
    data = np.load(name)
    print(data[:,0:3])
    ## Max rate
    # print(max(data[0:len(data),2]/data[0:len(data),1]))
#     print(data[np.argmax(data[0:len(data),2]/data[0:len(data),1]),1])

NameError: name 'np' is not defined

## Generate fake data

In [3]:
number_days = 60
time_interval = 2
data = np.zeros((200,3))

for day in range(number_days):
    time = 0
    for car in range(200):
        data[car,0] = time
        time = time + time_interval
        data[car,1] = 15
        data[car,2] = 4
    name = 'fake/data' + str(day)
    np.save(name, data)

In [10]:
# Test
day = 11
name = 'fake/data' + str(day) + '.npy'
data = np.load(name)
print(data[0,2])

4.0


## Generate real data for aggregate flexibility metrics

In [12]:
# Caltech
# Acquire data from database; Run once
# Database information
acn  = 'CaltechACN'
start = datetime(2018, 9,1)
end = datetime(2019, 9,1)
dt = timedelta(days = 1)
dates = np.arange(start, end, dt).astype(datetime)
day = 0
number_days= 0

# Generate data files
for date in dates:
        charging_start = date
        charging_end = date + timedelta(1)
        data = data_collection.get_data(acn, charging_start, charging_end, cond='ALL')
        number_days = number_days+1
        name = 'real_greedy/data' + str(day) + '.npy'
        np.save(name, data)
        day=day+1

# Polish data
for day in range(number_days):
    name = 'real_greedy/data' + str(day) + '.npy'
    data = np.load(name)
    for car in range(np.size(data,0)):
        # Eliminate over-night cars
        if data[car,0] + data[car,1] > 24:
            np.delete(data,car,0)
    # Sort according to arrivial time
    data =  data[data[:,0].argsort()]
    np.save(name, data)

In [13]:
# JPL
# Acquire data from database; Run once
# Database information
acn  = 'JplACN'
start = datetime(2018, 11,1)
end = datetime(2019, 9,1)
dt = timedelta(days = 1)
dates = np.arange(start, end, dt).astype(datetime)
day = 0
number_days= 0

# Generate data files
for date in dates:
    charging_start = date
    charging_end = date + timedelta(1)
    data = data_collection.get_data(acn, charging_start, charging_end, cond='ALL')
    number_days = number_days+1
    name = 'real_greedy_jpl/data' + str(day) + '.npy'
    np.save(name, data)
    day=day+1

# Polish data
for day in range(number_days):
    name = 'real_greedy_jpl/data' + str(day) + '.npy'
    data = np.load(name)
    for car in range(np.size(data,0)):
        # Eliminate over-night cars
        if data[car,0] + data[car,1] > 24:
            np.delete(data,car,0)
    # Sort according to arrivial time
    if len(data)!=0:
        data =  data[data[:,0].argsort()]
        np.save(name, data)

In [28]:
if len(data)==0:
    print('right')

right


In [27]:
# Test
day = 1
name = 'real_greedy/data' + str(day) + '.npy'
data = np.load(name)


name1 = '/Users/tonytiny/Documents/GitHub/aggregate_flexibility/results/aggregate_flexibility.npy'
test = np.load(name1)
print(data)

[[ 0.61222222 10.85333333  3.521     ]
 [ 2.4575     12.51083333 41.434     ]
 [ 3.5675      1.65638889  5.519     ]
 [ 7.01861111  2.06777778  2.257     ]
 [ 7.11861111  1.40111111  5.979     ]
 [ 7.33972222  1.00111111  6.668     ]
 [ 7.38694444  8.37166667  9.241     ]
 [ 8.68722222  1.46805556  2.743     ]
 [ 9.17333333  1.42722222  4.632     ]
 [10.84138889  2.28888889  7.48      ]
 [11.38083333  2.17666667  6.945     ]
 [11.38777778  1.11        7.284     ]
 [11.64666667  0.14666667  0.872     ]
 [12.08638889  3.34388889 11.376     ]
 [12.25222222  3.89027778  2.673     ]
 [12.50666667  1.40777778  9.548     ]
 [12.61305556  3.90027778 12.947     ]
 [13.11527778  5.24638889  6.603     ]
 [13.26611111  0.82861111  2.713     ]
 [13.26916667  7.37027778 13.238     ]
 [13.36472222  4.55277778 13.927     ]
 [13.38194444  4.72972222 13.884     ]
 [13.43638889  2.765       0.717     ]
 [13.6225     10.05416667 13.902     ]
 [13.82472222  2.0075     11.561     ]
 [13.92305556  2.23416667